In [58]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from tqdm import tqdm
import os
from PIL import Image
import cv2


In [59]:
class UCF101Dataset(Dataset):
    """UCF101 Dataset for video classification."""

    def __init__(self, video_dir, label_file, label_map, transform=None, keyframe_interval=10, num_frames=10):
        """
        Args:
            video_dir (string): Path to the directory with video frames or video clips.
            label_file (string): Path to the label file (e.g., train_labels.txt).
            label_map (dict): Mapping of class names to label indices.
            transform (callable, optional): Transform to be applied to each frame/clip.
            keyframe_interval (int, optional): Extract one frame every `keyframe_interval` frames.
            num_frames (int, optional): Fixed number of frames per video.
        """
        self.video_dir = video_dir
        self.transform = transform
        self.samples = []
        self.keyframe_interval = keyframe_interval
        self.num_frames = num_frames

        # Load video paths and labels from the label file
        with open(label_file, 'r') as file:
            for line in file:
                video_name, class_name = line.strip().split()
                video_path = os.path.join(video_dir, class_name, video_name)
                label = label_map[class_name]
                self.samples.append((video_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]

        # Load video frames
        frames = self.load_video_frames(video_path)

        # Truncate or pad frames to ensure a fixed number of frames
        if len(frames) > self.num_frames:
            frames = frames[:self.num_frames]
        elif len(frames) < self.num_frames:
            pad_size = self.num_frames - len(frames)
            padding = [torch.zeros_like(frames[0])] * pad_size
            frames.extend(padding)

        frames = torch.stack(frames)  # Combine frames into a single tensor

        return {'video': frames, 'label': label, 'path': video_path}

    def load_video_frames(self, video_path):
        """Load video frames at regular intervals."""
        import cv2
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_idx = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if frame_idx % self.keyframe_interval == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                if self.transform:
                    frame = self.transform(frame)
                frames.append(frame)
            frame_idx += 1
        cap.release()
        return frames


In [60]:
def custom_collate_fn(batch):
    """
    Custom collate function to handle varying frame counts in a batch.
    """
    videos = [item['video'] for item in batch]
    labels = [item['label'] for item in batch]
    paths = [item['path'] for item in batch]

    # Find the maximum number of frames in the batch
    max_frames = max(video.size(0) for video in videos)

    # Pad all videos to the maximum number of frames
    padded_videos = []
    for video in videos:
        pad_size = max_frames - video.size(0)
        if pad_size > 0:
            padding = torch.zeros((pad_size, *video.size()[1:]))
            video = torch.cat([video, padding], dim=0)
        padded_videos.append(video)

    padded_videos = torch.stack(padded_videos)
    labels = torch.tensor(labels)

    return {'video': padded_videos, 'label': labels, 'path': paths}


In [61]:
# Generate the label map
video_dir = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/'
class_names = sorted(os.listdir(video_dir))  # Folder names in the train directory
label_map = {class_name: idx for idx, class_name in enumerate(class_names)}

# Paths to label files
train_label_file = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/train_labels.txt'
val_label_file = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/val_labels.txt'
test_label_file = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/test_labels.txt'

# Create datasets
train_dataset = UCF101Dataset(
    video_dir=r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/', 
    label_file=train_label_file, 
    label_map=label_map,
    transform=transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    keyframe_interval=5  # Extract 1 frame every 10 frames
)

val_dataset = UCF101Dataset(
    video_dir=r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/val/', 
    label_file=val_label_file, 
    label_map=label_map,
    transform=transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    keyframe_interval=5
)

test_dataset = UCF101Dataset(
    video_dir=r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/test/', 
    label_file=test_label_file, 
    label_map=label_map,
    transform=transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    keyframe_interval=5
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=custom_collate_fn)

In [62]:
def verify_dataloader(dataloader, dataset_name):
    """
    Verify the correctness of the DataLoader by iterating over a few batches.

    Args:
        dataloader: DataLoader to verify.
        dataset_name: Name of the dataset being verified (train, val, test).
    """
    print(f"Verifying {dataset_name} DataLoader...")
    for i, batch in enumerate(dataloader):
        print(f"Batch {i + 1}:")
        print(f"Video Tensor Shape: {batch['video'].shape}")  # Should be (batch_size, num_keyframes, channels, height, width)
        print(f"Labels: {batch['label']}")  # Ensure labels are mapped correctly
        print(f"Paths: {batch['path']}")  # Check if file paths are correct

        # Stop after a few batches
        if i >= 2:
            break

    print(f"{dataset_name} DataLoader verified!\n")

# Verify train, val, and test loaders
verify_dataloader(train_loader, "Train")
verify_dataloader(val_loader, "Validation")
verify_dataloader(test_loader, "Test")


Verifying Train DataLoader...
Batch 1:
Video Tensor Shape: torch.Size([16, 10, 3, 128, 128])
Labels: tensor([27, 73, 41, 65, 89, 55, 33,  6, 15, 30, 62, 21,  6, 14, 26, 61])
Paths: ['C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/Fencing\\v_Fencing_g10_c02.avi', 'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/RockClimbingIndoor\\v_RockClimbingIndoor_g23_c02.avi', 'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/HorseRiding\\v_HorseRiding_g20_c03.avi', 'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/PlayingTabla\\v_PlayingTabla_g22_c04.avi', 'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/TableTennisShot\\v_TableTennisShot_g24_c02.avi', 'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/Nunchucks\\v_Nunchucks_g13_c06.avi', 'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/Haircut\\v_Haircut_g09_c05.avi', 'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/BaseballPitch\\

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
from sklearn.metrics import recall_score, accuracy_score

In [64]:
dtype = torch.FloatTensor  # Set dtype to FloatTensor for CPU

# Define Flatten layer
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size()  # Read in N, C, H, W
        return x.view(N, -1)  # Flatten the C * H * W values into a single vector per image
    
    
class Flatten3d(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)  # Flatten all dimensions except the batch size

# Define fixed model
num_classes = len(label_map)
# fixed_model_base = nn.Sequential(
#     nn.Conv2d(3, 8, kernel_size=7, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride=2),
#     nn.Conv2d(8, 16, kernel_size=7, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride=2),
#     Flatten(),
#     nn.ReLU(inplace=True),
#     nn.Linear(1936, num_classes)
# )

fixed_model_base = nn.Sequential(
    nn.Conv3d(in_channels=3, out_channels=64, kernel_size=2, stride=1),
    nn.ReLU(inplace=True),
    nn.MaxPool3d((1, 2, 2), stride=(1, 2, 2)),
    
    nn.Conv3d(in_channels=64, out_channels=256, kernel_size=(1, 3, 3), stride=1),
    nn.ReLU(inplace=True),
    nn.MaxPool3d((1, 3, 3), stride=(1, 2, 2)),

    nn.Dropout3d(0.1),
    Flatten3d(),

    nn.ReLU(inplace=True),
    nn.Linear(2073600, num_classes),  # Adjust dimensions based on the output of conv layers
    nn.LogSoftmax(dim=1)
)


fixed_model = fixed_model_base.type(dtype)  # Set model to the appropriate data type


# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(fixed_model.parameters(), lr=1e-4)  # Learning rate set to 1e-3



In [65]:
# def train(model, loss_fn, optimizer, train_loader, val_loader, num_epochs=10, print_every=100):
#     model.train()  # Set the model to training mode
#     for epoch in range(num_epochs):
#         print(f"Epoch {epoch + 1}/{num_epochs}")
#         running_loss = 0.0
#         for i, batch in enumerate(tqdm(train_loader, desc="Training")):
#             # Debugging: Check the structure of the batch
#             assert 'video' in batch and 'label' in batch, "Batch does not have 'video' or 'label' keys"
#             assert batch['video'].dim() == 5, "Video tensor should have 5 dimensions [batch_size, num_frames, 3, H, W]"
#             assert batch['label'].dim() == 1, "Label tensor should have 1 dimension [batch_size]"
# 
#             # Flatten the time dimension into the batch size
#             videos = batch['video']  # Shape: [batch_size, num_frames, 3, H, W]
#             batch_size, num_frames, C, H, W = videos.size()
#             x_var = videos.view(batch_size * num_frames, C, H, W).to(torch.float)  # Shape: [batch_size * num_frames, 3, H, W]
#             y_var = batch['label'].repeat_interleave(num_frames).to(torch.long)  # Repeat labels for all frames
# 
#             # Forward pass
#             scores = model(x_var)
#             loss = loss_fn(scores, y_var)
# 
#             # Backward pass
#             optimizer.zero_grad()
#             loss.backward()
# 
#             # Update weights
#             optimizer.step()
# 
#             running_loss += loss.item()
# 
#             if (i + 1) % print_every == 0:
#                 print(f"Batch {i + 1}, Loss: {loss.item():.4f}")
# 
#         # Validate the model
#         validate(model, val_loader)
# 
# # Updated validate function
# # Updated validate function
# def validate(model, val_loader):
#     model.eval()  # Set the model to evaluation mode
#     all_preds = []
#     all_labels = []
#     with torch.no_grad():
#         for batch in tqdm(val_loader, desc="Validation"):
#             assert 'video' in batch and 'label' in batch, "Batch does not have 'video' or 'label' keys"
#             videos = batch['video']  # Shape: [batch_size, num_frames, 3, H, W]
#             batch_size, num_frames, C, H, W = videos.size()
# 
#             # Correct usage of `.to()` with both device and dtype
#             x_var = videos.view(batch_size * num_frames, C, H, W).to(torch.device('cpu'), torch.float)
#             y_var = batch['label'].repeat_interleave(num_frames).long().to(torch.device('cpu'))
# 
#             scores = model(x_var)
#             _, preds = torch.max(scores, 1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(y_var.cpu().numpy())
# 
#     acc = accuracy_score(all_labels, all_preds)
#     recall = recall_score(all_labels, all_preds, average='macro')
#     print(f"Validation Accuracy: {acc * 100:.2f}%")
#     print(f"Validation Recall: {recall * 100:.2f}%")
# 
# # def test(model, test_loader):
# #     model.eval()  # Set the model to evaluation mode
# #     all_preds = []
# #     all_labels = []
# #     with torch.no_grad():
# #         for batch in tqdm(test_loader, desc="Testing"):
# #             x_var = Variable(batch['video'])
# #             y_var = Variable(batch['label']).long()
# #             scores = model(x_var)
# #             _, preds = torch.max(scores, 1)
# #             all_preds.extend(preds.cpu().numpy())
# #             all_labels.extend(y_var.cpu().numpy())
# #     acc = accuracy_score(all_labels, all_preds)
# #     recall = recall_score(all_labels, all_preds, average='macro')
# #     print(f"Test Accuracy: {acc * 100:.2f}%")
# #     print(f"Test Recall: {recall * 100:.2f}%")

In [66]:
def train(model, loss_fn, optimizer, train_loader, val_loader, num_epochs=10, print_every=10):
    model.train()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        running_loss = 0.0
        for i, batch in enumerate(tqdm(train_loader, desc="Training")):
            # Prepare input
            videos = batch['video']  # Shape: [batch_size, num_frames, 3, H, W]
            videos = videos.permute(0, 2, 1, 3, 4).to(torch.float32)  # Shape: [batch_size, 3, num_frames, H, W]
            labels = batch['label'].to(torch.long)

            # Forward pass
            scores = model(videos)
            loss = loss_fn(scores, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()

            # Update weights
            optimizer.step()

            running_loss += loss.item()
            if (i + 1) % print_every == 0:
                print(f"Batch {i + 1}, Loss: {loss.item():.4f}")

        # Validate the model
        #validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            videos = batch['video']
            videos = videos.permute(0, 2, 1, 3, 4).to(torch.float32)
            labels = batch['label'].to(torch.long)

            scores = model(videos)
            _, preds = torch.max(scores, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average='macro')
    print(f"Validation Accuracy: {acc * 100:.2f}%")
    print(f"Validation Recall: {recall * 100:.2f}%")

In [ ]:
# Train, validate, and test the model
train(fixed_model, loss_fn, optimizer, train_loader, val_loader, num_epochs=5, print_every=100)


Epoch 1/5


Training:  20%|██        | 100/498 [12:41<48:26,  7.30s/it] 

Batch 100, Loss: 3.9432


Training:  40%|████      | 200/498 [22:42<28:32,  5.75s/it]

Batch 200, Loss: 2.3896


...

Training:  20%|██        | 100/498 [11:04<48:16,  7.28s/it] 

Batch 100, Loss: 0.0104


Training:  40%|████      | 200/498 [23:48<38:48,  7.81s/it]

Batch 200, Loss: 0.0006


Training:  41%|████▏     | 206/498 [48:20<9:01:00, 111.16s/it] 

In [68]:
def test(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Test"):
            videos = batch['video']
            videos = videos.permute(0, 2, 1, 3, 4).to(torch.float32)
            labels = batch['label'].to(torch.long)

            scores = model(videos)
            _, preds = torch.max(scores, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average='macro')
    print(f"Validation Accuracy: {acc * 100:.2f}%")
    print(f"Validation Recall: {recall * 100:.2f}%")


test(fixed_model, test_loader)

Test: 100%|██████████| 171/171 [12:54<00:00,  4.53s/it]


Validation Accuracy: 71.24%
Validation Recall: 70.30%


In [69]:
from collections import deque
import torch

In [70]:
def load_label_map(label_file):
    """
    Load the label mapping from the label file.
    Args:
        label_file: Path to the label file.
    Returns:
        label_map: Dictionary mapping indices to action labels.
        reverse_map: Dictionary mapping action names to indices.
    """
    label_map = {}
    reverse_map = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_file, action_label = line.strip().split()
            if action_label not in reverse_map:
                idx = len(reverse_map)  # Assign a new index to unseen labels
                reverse_map[action_label] = idx
                label_map[idx] = action_label
    return label_map, reverse_map


In [71]:
label_file = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/test_labels.txt'
label_map, reverse_map = load_label_map(label_file)

In [72]:
def detect_action_realtime(
    model, label_map, device, video_source, output_file="output_video.mp4", frame_interval=10, frame_size=(64, 64)
):
    """
    Real-time action detection using a trained model with video output.

    Args:
    - model: The trained PyTorch model.
    - label_map: Dictionary mapping indices to action names.
    - device: The device ('cpu' or 'cuda') where the model will run.
    - video_source: Path to the video file or webcam index.
    - output_file: Path to save the output video with action labels.
    - frame_interval: Number of frames per segment to classify (e.g., 16).
    - frame_size: Tuple indicating the size to which video frames are resized.
    """
    model.eval()
    model.to(device)

    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error: Could not open video source.")
        return

    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Set up video writer for saving output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

    # Frame buffer and prediction history
    frame_buffer = deque(maxlen=frame_interval)
    prediction_history = deque(maxlen=5)  # Store the last 5 predictions

    # Transform to resize, convert to tensor, and normalize
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(frame_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Use tqdm to track the processing progress
    with tqdm(total=frame_count, desc="Processing video") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()

            # Check if the frame is valid
            if not ret or frame is None:
                print("Error: Frame is None. Skipping this frame.")
                break

            try:
                # Transform the frame
                resized_frame = transform(frame)
                frame_buffer.append(resized_frame)
            except Exception as e:
                print(f"Error in frame transformation: {e}")
                continue

            # Process frames if buffer is full
            if len(frame_buffer) == frame_interval:
                # Stack frames and adjust shape
                input_frames = torch.stack(list(frame_buffer), dim=0)  # Shape: (frame_interval, C, H, W)
                input_frames = input_frames.permute(1, 0, 2, 3).unsqueeze(0).to(device)  # Shape: (1, C, T, H, W)

                # Model prediction
                with torch.no_grad():
                    scores = model(input_frames)
                    _, preds = torch.max(scores, 1)

                    # Handle KeyError gracefully if predicted label is not in label_map
                    predicted_label = label_map.get(preds.item(), "Unknown Action")

                # Smooth predictions using history
                prediction_history.append(predicted_label)
                smoothed_prediction = max(set(prediction_history), key=prediction_history.count)

                # Display the smoothed prediction on the frame
                cv2.putText(frame, f"Action: {smoothed_prediction}", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # Write the frame to the output video
            out.write(frame)
            pbar.update(1)

            # Show the video in real-time
            cv2.imshow("Real-Time Action Detection", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define label map
video_dir = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new/dataset/train/'
class_names = sorted(os.listdir(video_dir))  # Folder names in the train directory
label_map = {idx: class_name for idx, class_name in enumerate(class_names)}

# Paths for input and output videos
video_file = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new3/dataset/test4.avi'
output_file = r'C:/Users/22322/Desktop/UIUC/ECE549/final/new/new3/dataset/output_video4.mp4'

# Run detection
detect_action_realtime(
    model=fixed_model,
    label_map=label_map,
    device=device,
    video_source=video_file,
    output_file=output_file,
    frame_interval=10,
    frame_size=(128, 128)
)

Processing video: 100%|██████████| 116/116 [00:10<00:00, 10.61it/s]


Error: Frame is None. Skipping this frame.
